# Problem 4: Model selection for GMM with BIC and cross validation

In many machine learning applications model selection is crucial. In this exercise, you
will practice two common approaches for model selection: 

- Bayesian Information Criterion (BIC) (as an approximation to ‘Bayesian model selection’) and  

- Cross-Validation (as a representative for a predictive model selection criterion).

You are given a data set (1000 samples of dimension 2) contained in the file data.pickle, which has been sampled from a Gaussian Mixture Model (GMM) using three classes(the true class labels are given for your convenience, but they should not be used in learning the model).

In the given code template below, the data will be divided into training and test sets.

**(a)** Complete the functions 'compute_bic' and 'cross_validate'. Use both criteria to select the number of components in the GMM using the training data. Plot the both the BIC and the validation log-likelihoods as a function of the number of components, as well as the data with the best model. Do both methods find a model with three components as the most likely?

**(b)** Use the selected models to evaluate the test set log-likelihood.

**(c)** Explain briefly the pros and cons of the two approaches and comment which approach you would consider better and why.

**Hint**: What is the total number of parameters needed to specify the component
means and covariance matrices, and the mixture weights? You will need this
number to compute BIC.

In [ ]:
# This Starter code for problem 4. The solution template is in the cell below.
# Tools for learning Gaussian mixture models; adapted from the BRMLtoolkit by David Barber.
import numpy as np
import numpy.matlib as matlib
import numpy.linalg as LA
#import scipy.misc
from scipy.special import logsumexp

def condp(X):
    return X / np.sum(X, axis=0)

def condexp(logp):
    pmax = np.max(logp, axis=0)
    P = logp.shape[0]
    return condp(np.exp(logp - np.tile(pmax, (P, 1))))

def GMMlogp(X, H, P, m, S):
    D, N = X.shape  # dimension and number of points

    logp = np.zeros((N, H))
    for i in range(H):
        invSi = LA.inv(S[i,:,:])
        sign, logdetSi = LA.slogdet(2 * np.pi * S[i,:,:])

        for n in range(N):
            v = X[:,n] - m[:,i]
            logp[n,i] = -0.5 * (v @ invSi @ v) - 0.5 * logdetSi + np.log(P[i])

    return logp



# Log Likelihood of data X under a Gaussian Mixture Model
#
# X : each column of X is a datapoint.
# P : mixture coefficients
# m : means
# S : covariances
#
# Returns: A list containing the log likelihood for each data point in X
def GMMloglik(X, P, m, S):
    N = X.shape[1]
    H = m.shape[1]

    logp = GMMlogp(X, H, P, m, S)
    logl = [logsumexp(a=logp[n,:], b=np.ones(H)) for n in range(N)]

    return logl

# Fit a mixture of Gaussian to the data X using EM
#
# X : each column of X is a datapoint.
# H : number of components of the mixture.
# n_iter : number of EM iterations
#
# Returns: (P, m, S, loglik, phgn)
# P : learned mixture coefficients
# m : learned means
# S : learned covariances
# loglik : log likelihood of the learned model
# phgn : mixture assignment probabilities
def GMMem(X, H, n_iter):
    D, N = X.shape  # dimension and number of points

    # initialise the centres to random datapoints
    r = np.random.permutation(N)
    m = X[:, r[:H]]

    # initialise the variances to be large
    s2 = np.mean(np.diag(np.cov(X)))
    S = matlib.tile(s2 * np.eye(D), [H, 1, 1])

    # intialise the component probilities to be uniform
    P = np.ones(H) / H

    for emloop in range(n_iter):
        # E-step:
        logpold = GMMlogp(X, H, P, m, S)

        phgn = condexp(logpold.T)  # responsibilities
        pngh = condp(phgn.T)       # membership

        # M-step:
        for i in range(H):   # now get the new parameters for each component
            tmp = (X - np.tile(m[:,i:i+1], N)) * np.tile(np.sqrt(pngh[:,i]), (D,1))
            Scand = np.dot(tmp, tmp.T)

            if LA.det(Scand) > 0.0001:   # don't accept too low determinant
                S[i,:,:] = Scand

        m = np.dot(X, pngh)
        P = np.sum(phgn, axis=1) / N

    logl = [logsumexp(a=logpold[n,:], b=np.ones(H)) for n in range(N)]
    logl = np.sum(logl)

    return P, m, S, logl, phgn

In [ ]:
# Template for problem 4
import matplotlib.pyplot as plt
import pickle

np.random.seed(0)

def compute_bic(Xtrain, totalComponents):
    BICs = []
    for H in range(1, totalComponents+1):    # number of mixture components
        print("H: {}".format(H))

        P, m, S, loglik, phgn = GMMem(Xtrain, H, 100)  # fit to data
        
        # Use BIC formula described in lecture notes (without multiplying with -2)
        # numParams = ?     # number of parameters in the model
        # BIC = ?           # BIC for the model
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        BICs.append(BIC)
    return BICs

def cross_validate(Xtrain, totalComponents):
    foldCount = 5    # number of folds

    loglik = np.zeros((totalComponents, foldCount))

    Nlearning = Xtrain.shape[1]
    order = np.random.permutation(Nlearning)    # to randomize the sample order

    for H in range(1, totalComponents+1):     # number of mixture components
        print("H: {}".format(H))

        for fold in range(foldCount):    # K-fold cross validation (K=5)
            ind = fold * int(Nlearning/foldCount) + np.arange(int(Nlearning/foldCount))
            val_indices = order[ind]

            training_indices = np.setdiff1d(np.arange(Nlearning), val_indices);

            X_train = Xtrain[:,training_indices]  # cv training data
            X_val   = Xtrain[:,val_indices]       # cv validation data

            # train model
            P1, m1, S1, loglik1, phgn1 = GMMem(X_train, H, 100)   # fit model

            # Predict using the cv trained model
            # logl1 = ?
            # loglik[H-1,fold] = ?
            
            # YOUR CODE HERE
            raise NotImplementedError()
            
    return loglik

def model_selection(Xtrain, totalComponents, criterion_flag):
    np.random.seed(0)
    if criterion_flag:
        print('Computing BIC Score')
        scores = compute_bic(Xtrain, totalComponents)
        ylabel = "(BIC)"
    elif not criterion_flag:
        print('Computing Cross Validation Score')
        scores = cross_validate(Xtrain, totalComponents)
        scores = np.mean(scores, axis=1)
        ylabel = "(Cross Validation)"

    # plot the model selection curve
    plt.bar(np.arange(1, totalComponents+1), scores)
    plt.xlabel('Number of Mixture Components')
    plt.ylabel(ylabel)
    plt.title('Model Selection' + ylabel)
    plt.show()

    # select the number of mixture components which minimizes the model selection criteria
    h = np.argmax(scores) + 1
    print('Selected Number of Mixture Components = {}'.format(h))
    
    return h

# Now train full model with selected number of mixture components
def train_full_model(Xtrain, Xtest, h):
    np.random.seed(0)
    P, m, S, loglik, phgn = GMMem(Xtrain, h, 100)  # fit to data

    # Predict using the full trained model (Use GMMem.GMMloglik)
    # logl = ?
    # print('Test Data Log Likelihood = {0:f}'.format(?))

    # YOUR CODE HERE
    raise NotImplementedError()

    # Plot the best GMM model
    plot_data()

    for i in range(h):
        dV, E = LA.eig(S[i,:,:])

        theta = np.arange(0, 2*np.pi, 0.1)
        p = np.sqrt(dV.reshape(D,1)) * [np.cos(theta), np.sin(theta)]
        x = (E @ p) + np.tile(m[:,i:i+1], (1, len(theta)))

        plt.plot(x[0], x[1], 'r-', linewidth=2)

    plt.title('Training data, Test data (in black)')
    plt.show()
    

# load data
with open("/coursedata/data.pickle", "rb") as f:
    X, labels = pickle.load(f)

D, N = X.shape   # dimension and number of data points

ratio = 0.75
train_ind = np.random.choice(N, int(ratio * N), replace=False)   # training data index
test_ind = np.setdiff1d(np.arange(N), train_ind)                 # test data index

Xtrain = X[:,train_ind]            # training data
Xtrain_labels = labels[train_ind]  # training data labels

Xtest = X[:,test_ind]            # test data
Xtest_labels = labels[test_ind]  # test data labels

# plot training and test data
def plot_data():
    for i in sorted(set(Xtrain_labels)):
        X_comp = Xtrain[:, Xtrain_labels == i]
        plt.plot(X_comp[0], X_comp[1], '.' + 'brgmcyk'[i-1], markersize=6)

    plt.plot(Xtest[0], Xtest[1], 'kd', markersize=4, markeredgewidth=0.5, markerfacecolor="None")

plot_data()
plt.title('Training data, Test data (in black)')
plt.show()

In [ ]:
np.random.seed(0)
totalComponents = 5  # max number of mixture components
criterion_flag = 0   # 0: cross validation, 1: BIC

h = model_selection(Xtrain, totalComponents, criterion_flag)
train_full_model(Xtrain, Xtest, h)

In [ ]:
np.random.seed(0)
totalComponents = 5  # max number of mixture components
criterion_flag = 1   # 0: cross validation, 1: BIC

h = model_selection(Xtrain, totalComponents, criterion_flag)
train_full_model(Xtrain, Xtest, h)